In [17]:
from music21 import *
import pretty_midi as pm
import numpy as np
rn = np.random.default_rng
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/musescore'
us['musicxmlPath'] = '/usr/bin/musescore'

In [6]:
s = converter.parse("ode2joy.musicxml")
# s = converter.parse("Untitled_score-Piano.musicxml").flatten().chordify()
# s.show()
fp = s.write('midi', 'ode2joy.midi')
midi = pm.PrettyMIDI('ode2joy.midi')

In [63]:
# for c in midi.instruments[0].notes:
#     print(c)

In [59]:
def randomize_note_times(midi, mean_shift, stdev_shift, mean_dur_var, stdev_dur_var):
    for inst in midi.instruments:
        # Shift note times
        for note in inst.notes:
            shift = rn().normal(loc=mean_shift, scale=stdev_shift)
            dur_var = rn().normal(loc=mean_dur_var, scale=stdev_dur_var)
            dur = note.end - note.start
            start = note.start - shift
            end = start + max(0, (dur + dur_var))
            note.start = start
            note.end = end
        # Make sure note times are normalized
        min_time = float('inf')
        for note in inst.notes:
            if note.start < min_time:
                min_time = note.start
        if min_time < 0:
            for note in inst.notes:
                note.start += min_time
                note.end += min_time

def add_pitch_bends(midi, lambda_occur, mean_delta, stdev_delta, step_size):
    for inst in midi.instruments:
        inst.pitch_bends = []
        # Flatten note times list
        single_notes = []
        last_time = 0.0
        for note in inst.notes:
            if note.start > last_time:
                single_notes.append(note)
                last_time = note.end

        fixed_bend_points = []
        # Add fixed point pitch bends
        for note in single_notes:
            # Do 1 pitch bend at start of each note
            bend = int(rn().normal(mean_delta, stdev_delta))
            if bend > 8192:
                bend = 8192
            elif bend < -8191:
                bend = -8191
            fixed_bend_points.append(pm.PitchBend(bend, note.start))
            # Add more randomly
            occurrences = rn().poisson(lam=lambda_occur*(note.end-note.start))
            for i in range(occurrences):
                time = rn().uniform(low=note.start, high=note.end)
                bend = int(rn().normal(mean_delta, stdev_delta))
                fixed_bend_points.append(pm.PitchBend(bend, time))

        # Sort by time from least to greatest
        fixed_bend_points.sort(key=(lambda x : x.time))

        # Linear interpolation
        inst.pitch_bends = fixed_bend_points.copy()
        for i in range(len(fixed_bend_points) - 1):
            l_pb = fixed_bend_points[i]
            r_pb = fixed_bend_points[i+1]
            n_points_to_add = int(np.floor((r_pb.time - l_pb.time) / step_size))
            for j in range(1, n_points_to_add+1):
                time = l_pb.time + j*step_size
                bend = int(l_pb.pitch + ((r_pb.pitch - l_pb.pitch) * (j / (n_points_to_add + 1))))
                if bend > 8192:
                    bend = 8192
                elif bend < -8191:
                    bend = -8191
                inst.pitch_bends.append(pm.PitchBend(bend, time))    

In [64]:
midi = pm.PrettyMIDI('ode2joy.midi')
randomize_note_times(midi=midi, mean_shift=0, stdev_shift=0.04, mean_dur_var=0, stdev_dur_var=0.08)
add_pitch_bends(midi=midi, lambda_occur=2, mean_delta=0, stdev_delta=500, step_size=0.01)
midi.write('o2j_modified.midi')

In [5]:
# for n in s.recurse():
#     if type(n) == chord.Chord:
#         for my_note in n:
#             print(my_note.pitch)